In [3]:
import sagemaker

In [1]:
!pip install --upgrade tensorflow ktrain

     |████████████████████████████████| 516.2MB 11kB/s s eta 0:00:01   |▌                               | 7.4MB 11.5MB/s eta 0:00:45     |██████████████                  | 225.6MB 56.4MB/s eta 0:00:06     |███████████████████▊            | 318.7MB 52.6MB/s eta 0:00:04
  Using cached https://files.pythonhosted.org/packages/12/11/49bdde1b08a210365c04367e84d3fb1489db6f4b262358a10719962891a2/ktrain-0.16.3.tar.gz
     |████████████████████████████████| 460kB 48.7MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 43.6MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 41.8MB/s eta 0:00:01
     |████████████████████████████████| 6.7MB 43.8MB/s eta 0:00:01     |█▎                              | 266kB 43.8MB/s eta 0:00:01     |████████████████████████████▏   | 5.9MB 43.8MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 41.3MB/s eta 0:00:01
     |████████████████████████████████| 983kB 30.5MB/s eta 0:00:01
     |████████████████████████████████| 19.2MB 38.4MB

In [4]:
!pwd

/home/ec2-user/SageMaker/Yelp-ds/sentiment/train


In [10]:
# sagemaker.s3.S3Downloader.download("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/ktrain/trn.p", "./")

In [11]:
# sagemaker.s3.S3Downloader.download("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/ktrain/val.p", "./")

In [12]:
# sagemaker.s3.S3Downloader.download("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/ktrain/preproc.p", "./")

In [5]:
import pickle
import time

In [6]:
start = time.time()
trn = pickle.load(open( "trn.p", "rb" ))
val = pickle.load(open( "val.p", "rb" ))
preproc = pickle.load(open( "preproc.p", "rb" ))

print(time.time() - start)
print(type(trn), type(val), type(preproc))

13.893087148666382
<class 'ktrain.text.preprocessor.TransformerDataset'> <class 'ktrain.text.preprocessor.TransformerDataset'> <class 'ktrain.text.preprocessor.DistilBertPreprocessor'>


In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [8]:
import ktrain
from ktrain import text

In [9]:
model = text.text_classifier('distilbert', train_data=trn, preproc=preproc)

Is Multi-Label? False
maxlen is 256



done.


In [10]:
learner = ktrain.get_learner(model,
                             train_data=trn, 
                             val_data=val, 
                             batch_size=6)

In [ ]:
learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
 7606/53008 [===>..........................] - ETA: 54:03 - loss: 0.2069 - accuracy: 0.9192

In [ ]:
learner.validate(val_data=val)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.save('./models/ktrain/distilbert-pos-neg-3-is-pos')